In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [3]:
train_dataset = ImageFolder('/home/radhakrishnan.39a/Documents/NN/Sentinel_Dataset/val/',transform=transforms.ToTensor())

test_dataset = ImageFolder('/home/radhakrishnan.39a/Documents/NN/Sentinel_Dataset/test/',transform=transforms.ToTensor())

trainloader = DataLoader(train_dataset, batch_size=100,
                            num_workers=0, drop_last=False, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=512,
                            num_workers=0, drop_last=False, shuffle=False)

In [4]:
print(trainloader.dataset.classes)

['clear', 'cloudy']


In [5]:
device = torch.device('cuda:1')
model = models.resnet18(pretrained=False)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
# for param in model.parameters():
#     param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(512, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 2),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
train_dataset = ImageFolder('/home/radhakrishnan.39a/Documents/NN/Sentinel_Dataset/train_5000/',transform=transforms.ToTensor())

test_dataset = ImageFolder('/home/radhakrishnan.39a/Documents/NN/Sentinel_Dataset/test/',transform=transforms.ToTensor())

trainloader = DataLoader(train_dataset, batch_size=100,
                            num_workers=0, drop_last=False, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=512,
                            num_workers=0, drop_last=False, shuffle=False)
device = torch.device('cuda:1')
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(nn.Linear(512, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 2),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)
epochs = 80
steps = 0
running_loss = 0
print_every = 100
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            tp = 0
            fp = 0
            tn = 0
            fn = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f} "
                 )
            running_loss = 0
            model.train()
accuracy = 0
tp = 0
fp = 0
tn = 0
fn = 0
model.eval()
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        prediction = torch.argmax(ps, dim=1)
        for i in range(len(prediction.int())):
            if ((prediction[i] == 0) and (labels.data[i] == 0)):
                tp +=1
            elif ((prediction[i] == 0) and (labels.data[i] == 1)):
                fp+=1
                #fpl.append(fnames[i])
            elif ((prediction[i] == 1) and (labels.data[i] == 1)):
                tn+=1
                #tnl.append(fnames[i])
            elif ((prediction[i] == 1) and (labels.data[i] == 0)):
                fn+=1
recall = tp/(tp+fn)
precision = tp/(tp+fp)
f1_score = (2 * precision * recall) / (precision + recall)
print(f"Final Epoch"
      f"\nTest accuracy: {accuracy/len(testloader):.3f} "
      f"Test precision: {precision:.3f} "
      f"Test recall: {recall:.3f} "
      f"Test F1: {f1_score:.3f} "
     )
torch.save(model, 'baseline.pth')